In [19]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.preprocessing import StandardScaler
from scipy import stats as s
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
import sklearn.cross_validation as crv
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
import random
from sklearn.metrics import mean_squared_error

In [20]:
def col_name():
    p0 = ['x','y','z']
    marker =[]
    p2 =[]
    for a in range(1,44):       #Mocapマーカー数:43個
        p1='m'+ str(a)
        for b in p0:
            p2 = p1 + b
            marker.append(p2)
    return marker

def read_mocap(file):
    data_mocap = pd.read_csv(file,sep=' ',header=None)
    #座標値列の列名変更
    data_mocap=data_mocap.rename(columns={data_mocap.columns[a]:col_name()[a] for a in range(129)} )
    #座標値列以外の列名変更
    data_mocap=data_mocap.rename(columns={129:'FrameNumber',130:'TimeStamp'})
    return data_mocap

#fix NaN
def fix_NaN(data):
    for  c  in  data.columns:
        col = data[c]  #'X'列から1列ずつ見る
        for r in range(len(data)):  #1行ずつ見る
            if col[r] == 0.0:       #0値が見つかれば
                data.iloc[r][c] = 'NaN' #'NaN'を代入
    mc_clean = data.interpolate()
    mc_clean = mc_clean.fillna(0)
    return mc_clean

def obtain_statistical_feature(window):
    #features = np.array(np.array(np.var(window, axis=0)))
    #features=np.array(np.array(np.mean(window-np.mean(window), axis=0)))
    features=np.array( np.array(np.mean(window-np.median(window), axis=0))) 
    features=np.append(features, np.array(np.std(window, axis=0))) 
    features=np.append(features, np.array(np.var(window, axis=0)))
    features=np.append(features, np.array(s.skew(window, axis=0)))
    features=np.append(features, np.array(s.kurtosis(window, axis=0)))
    features=np.append(features, np.array(get_tw_col_var(window)))
    return features.reshape(1,len(features))


def get_tw_variance(window):
    total_cols = window.shape[1]
    tw_var = []
    for i in range (total_cols):
        column = window.iloc[:,i]
        tw_var.append(get_tw_col_var(column))
    return tw_var


def get_tw_col_var(column):
    mean = np.mean(column)
    total = len(column)
    
    v_sum = 0
    for i in range(total):
        w = np.exp(-0.5*(total-i))
        #print("el peso", w, "indice", i, "total", total)
        v = w*np.square(column[i]-mean)
        v_sum += v
    return v_sum

#速度加速度を求める関数を定義
def cal_va(data):
    diffs = pd.DataFrame()
    for i in range(len(data)-1):
        diff = data.iloc[i+1]-data.iloc[i]
        diffs = diffs.append(diff,ignore_index=True)
        
    vs = pd.DataFrame()
    for i in range(len(diffs)):
        v = diffs.iloc[i][1:]/diffs.iloc[i]["TimeStamp"]
        vs = vs.append(v,ignore_index=True)
    return vs




In [21]:
def read_acc(file):
    data_acc = pd.read_csv(file,sep=' ',header=None)
   
    data_acc = data_acc.rename(columns={data_acc.columns[0]:'X_acc', data_acc.columns[1]:'Y_acc', data_acc.columns[2]:'Z_acc',data_acc.columns[3]:'TimeStamp'} )
    
    return data_acc



#線形加速度を得る
def linear_acc(data):
    alpha = 0.9
    for  c  in  ['X_acc','Y_acc','Z_acc']:
        g = 0  
        col = data[c]           #'X'列から1列ずつ見る
        for r in range(len(data)):         #1行ずつ見る
            #Isolate the force of gravity with the low-pass filter.
            g = alpha*g + (1-alpha) * col[r]
            #Remove the gravity contribution with the high-pass filter.
            linear = col[r] - g
            data.iloc[r][c] = linear
        linear_acc = data
    return linear_acc

In [22]:
def one_acc1(data):
    xyz = pd.DataFrame(columns=["acc"])
    accs = []
    for i in range(len(data)):
        #ベクトルの大きさを求める
        acc = np.linalg.norm(data.loc[i])
        accs.append(acc)
    xyz["acc"]= accs
    #xyz["TimeStamp"] = data["TimeStamp"].tolist()
    return xyz

In [23]:
def one_acc2(data):
    xyz = pd.DataFrame(columns=["acc"])
    accs = []
    for i in range(len(data)):
        #ベクトルの大きさを求める
        acc = np.linalg.norm(data.loc[i][data.columns != "TimeStamp"])
        accs.append(acc)
    xyz["acc"]= accs
    #xyz["TimeStamp"] = data["TimeStamp"].tolist()
    return xyz

In [24]:
def features(f):
    x = np.empty([0,2])
    i = 0
    while i < len(f)-1:
        k = np.array(f["acc"][i])
        k = np.append(k,f["acc"][i+1])
        k = k.reshape(1,len(k))
        x =np.append(x,k,axis=0)
        i +=1
    return x

def my_index_multi(l, x):
    return [i for i, _x in enumerate(l) if _x == x]

In [25]:
def main_functuion(sampling_rate,marker,sensor):
    x = np.empty([0,2])
    y = np.empty([0,])
    random.seed(0)
    print("marker:",marker)
    print("sensor:",sensor)
    #Read all mocap data and acc data
    #subjects =  ["01","02","03","04","05","06","07","08","09","10","11","12"]
    subjects = random.sample( ["01","02","03","04","05","06","07","08","09","10","11","12"],8)
    actions = ["01","02","03","04","05","06","07","08","09","10","11"]
   
    for subject in subjects:
        for action in actions:
            for record in range(1,6):
            #for record in random.sample( [1,2,3,4,5],2):    
                try:
                    data = read_mocap('/Users/takeshin/odrive/sozolab_gdrive/students/Shingo/BerkeleyMHAD/Mocap/OpticalData/moc_s'+str(subject)+'_a'+str(action)+'_r0'+str(record)+'.txt')
                    print('moc_s'+str(subject)+'_a'+str(action)+'_r0'+str(record)+'.txt')
                    columns = ['m'+str(marker)+'x','m'+str(marker)+'y','m'+str(marker)+'z']
                    cols = np.array(columns)
                    cols = list(np.ravel(cols))
                    cols.append("TimeStamp")
                    data = data[cols]
                    data = fix_NaN(data)
                    
                    i = 0
                    mc_ds = pd.DataFrame()
                    #処理
                    while  i < (len(data)//(480//sampling_rate))*(480//sampling_rate):
                        mc_ds = mc_ds.append(data.iloc[i : i+(480//sampling_rate),:].mean(),ignore_index=True)
                        i+=(480//sampling_rate)
                    if i < len(data):
                        mc_ds = mc_ds.append(data.iloc[i :,:].mean(),ignore_index=True)
                    
                    
                    v = cal_va(mc_ds)
                    times = mc_ds['TimeStamp'].shift(periods=-1)
                    mc_v = pd.DataFrame(v)
                    mc_v["TimeStamp"]=times
                    mc_v = mc_v.dropna()

                    
                    a = cal_va(mc_v)
                    times = mc_v['TimeStamp'].shift(periods=-1)
                    mc_a = pd.DataFrame(a)
                    mc_a["TimeStamp"]=times
                    mc_a = mc_a.dropna()
                    #print(mc_a)                                           
                
                    
                    #単位を[G]に変換  [mm/s^2] →  [G]
                    accel = mc_a.loc[:, mc_a.columns != 'TimeStamp']*0.001/9.80665
                    accel = accel.rolling(window= 10, center=False).mean()
                    accel = accel.dropna()
                    accel = accel.reset_index()
                    #print(accel)
                    v_a = one_acc1(accel)
                    #print(v_a)
                    
                   
                    

                    #リアル加速度データの読み込み
                    r_a = read_acc('/Users/takeshin/odrive/sozolab_gdrive/students/Shingo/BerkeleyMHAD/Accelerometer/Shimmer0'+str(sensor)+'/acc_h0'+str(sensor)+'_s'+str(subject)+'_a'+str(action)+'_r0'+str(record)+'.txt')
                    print('acc_h0'+str(sensor)+'_s'+str(subject)+'_a'+str(action)+'_r0'+str(record)+'.txt')
                    r_a = fix_NaN(r_a)
                    r_a = linear_acc(r_a) 
                    r_a = one_acc2(r_a)
                    #print(r_a)
                    
                    k = features(v_a)
                    #print(x)
                    
                    #処理
                    if len(k) <= len(r_a):
                        x = np.append(x,k,axis =0)
                        y = np.append(y,r_a["acc"][1:len(k)+1], axis=0)
                        #y = np.append(y,r_a["acc"][0:len(k)], axis=0)
                    else:
                        #x = np.append(x,k[0:len(r_a)],axis =0)
                        # y = np.append(y,r_a["acc"], axis=0)
                        x = np.append(x,k[0:len(r_a)-1],axis =0)
                        y = np.append(y,r_a["acc"][1:], axis=0)
                        
                    
                except FileNotFoundError:
                    pass
                    
    return x, y

In [26]:
x,y = main_functuion(30,30,3)   

marker: 30
sensor: 3
moc_s07_a01_r01.txt
acc_h03_s07_a01_r01.txt
moc_s07_a01_r02.txt
acc_h03_s07_a01_r02.txt
moc_s07_a01_r03.txt
acc_h03_s07_a01_r03.txt
moc_s07_a01_r04.txt
acc_h03_s07_a01_r04.txt
moc_s07_a01_r05.txt
acc_h03_s07_a01_r05.txt
moc_s07_a02_r01.txt
acc_h03_s07_a02_r01.txt
moc_s07_a02_r02.txt
acc_h03_s07_a02_r02.txt
moc_s07_a02_r03.txt
acc_h03_s07_a02_r03.txt
moc_s07_a02_r04.txt
acc_h03_s07_a02_r04.txt
moc_s07_a02_r05.txt
acc_h03_s07_a02_r05.txt
moc_s07_a03_r01.txt
acc_h03_s07_a03_r01.txt
moc_s07_a03_r02.txt
acc_h03_s07_a03_r02.txt
moc_s07_a03_r03.txt
acc_h03_s07_a03_r03.txt
moc_s07_a03_r04.txt
acc_h03_s07_a03_r04.txt
moc_s07_a03_r05.txt
acc_h03_s07_a03_r05.txt
moc_s07_a04_r01.txt
acc_h03_s07_a04_r01.txt
moc_s07_a04_r02.txt
acc_h03_s07_a04_r02.txt
moc_s07_a04_r03.txt
acc_h03_s07_a04_r03.txt
moc_s07_a04_r04.txt
acc_h03_s07_a04_r04.txt
moc_s07_a04_r05.txt
acc_h03_s07_a04_r05.txt
moc_s07_a05_r01.txt
acc_h03_s07_a05_r01.txt
moc_s07_a05_r02.txt
acc_h03_s07_a05_r02.txt
moc_s07_a05

KeyboardInterrupt: 

In [9]:
print(x.shape)
print(y.shape)

(89536, 2)
(89536,)


In [10]:
#変換モデル 回帰
clf2 = RandomForestRegressor()
#clf2 = LinearRegression()
clf2.fit(x, y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [11]:
def main_functuion2(sampling_rate,*markers):
    X = np.empty([0,6*len(markers)])
    y = np.empty([0,])
    random.seed(0)
    #Read all mocap data
    b = random.sample( ["01","02","03","04","05","06","07","08","09","10","11","12"],8)
    a =  ["01","02","03","04","05","06","07","08","09","10","11","12"]
    subjects = list(set(a) - set(b))
    #subjects =  ["01","02","03","04","05","06","07","08","09","10","11","12"]
    #subjects =  ["07","08","09","10","11","12"]
    actions = ["01","02","03","04","05","06","07","08","09","10","11"]
    for subject in subjects:
        for action in actions:
            for record in range(1,6):
            #for record in range(4,6):    
                try:
                    data = read_mocap('/Users/takeshin/odrive/sozolab_gdrive/students/Shingo/BerkeleyMHAD/Mocap/OpticalData/moc_s'+str(subject)+'_a'+str(action)+'_r0'+str(record)+'.txt')
                    print('moc_s'+str(subject)+'_a'+str(action)+'_r0'+str(record)+'.txt')
                    columns = [['m'+str(i)+'x','m'+str(i)+'y','m'+str(i)+'z'] for i in markers]
                    cols = np.array(columns)
                    cols = list(np.ravel(cols))
                    cols.append("TimeStamp")
                    data = data[cols]
                    data = fix_NaN(data)
                    
                    i = 0
                    mc_ds = pd.DataFrame()
                    #処理
                    while  i < (len(data)//(480//sampling_rate))*(480//sampling_rate):
                        mc_ds = mc_ds.append(data.iloc[i : i+(480//sampling_rate),:].mean(),ignore_index=True)
                        i+=(480//sampling_rate)
                    if i < len(data):
                        mc_ds = mc_ds.append(data.iloc[i :,:].mean(),ignore_index=True)
                
                    
                    v = cal_va(mc_ds)
                    times = mc_ds['TimeStamp'].shift(periods=-1)
                    mc_v = pd.DataFrame(v)
                    mc_v["TimeStamp"]=times
                    mc_v = mc_v.dropna()

                    
                    a = cal_va(mc_v)
                    times = mc_v['TimeStamp'].shift(periods=-1)
                    mc_a = pd.DataFrame(a)
                    mc_a["TimeStamp"]=times
                    mc_a = mc_a.dropna()
                    #print(mc_a)                                           
                
                    
                    #単位を[G]に変換  [mm/s^2] →  [G]
                    accel = mc_a.loc[:, mc_a.columns != 'TimeStamp']*0.001/9.80665
                    accel = accel.rolling(window= 10, center=False).mean()
                    accel = accel.dropna()
                    accel = accel.reset_index()
                    #print(accel)
                    v_a = one_acc1(accel)
                    k = features(v_a)
                    #print(k)
                    
                    a_predict = clf2.predict(k).tolist()
                    #print(a_predict)
                    
                    sf = obtain_statistical_feature(a_predict)
                    X =np.append(X,sf, axis=0)
                    y = np.append(y,[action], axis=0)
                
                
                except FileNotFoundError:
                    pass
                    
    return X, y

In [12]:
x2,y2= main_functuion2(30,30)

moc_s06_a01_r01.txt
moc_s06_a01_r02.txt
moc_s06_a01_r03.txt
moc_s06_a01_r04.txt
moc_s06_a01_r05.txt
moc_s06_a02_r01.txt
moc_s06_a02_r02.txt
moc_s06_a02_r03.txt
moc_s06_a02_r04.txt
moc_s06_a02_r05.txt
moc_s06_a03_r01.txt
moc_s06_a03_r02.txt
moc_s06_a03_r03.txt
moc_s06_a03_r04.txt
moc_s06_a03_r05.txt
moc_s06_a04_r01.txt
moc_s06_a04_r02.txt
moc_s06_a04_r03.txt
moc_s06_a04_r04.txt
moc_s06_a04_r05.txt
moc_s06_a05_r01.txt
moc_s06_a05_r02.txt
moc_s06_a05_r03.txt
moc_s06_a05_r04.txt
moc_s06_a05_r05.txt
moc_s06_a06_r01.txt
moc_s06_a06_r02.txt
moc_s06_a06_r03.txt
moc_s06_a06_r04.txt
moc_s06_a06_r05.txt
moc_s06_a07_r01.txt
moc_s06_a07_r02.txt
moc_s06_a07_r03.txt
moc_s06_a07_r04.txt
moc_s06_a07_r05.txt
moc_s06_a08_r01.txt
moc_s06_a08_r02.txt
moc_s06_a08_r03.txt
moc_s06_a08_r04.txt
moc_s06_a08_r05.txt
moc_s06_a09_r01.txt
moc_s06_a09_r02.txt
moc_s06_a09_r03.txt
moc_s06_a09_r04.txt
moc_s06_a09_r05.txt
moc_s06_a10_r01.txt
moc_s06_a10_r02.txt
moc_s06_a10_r03.txt
moc_s06_a10_r04.txt
moc_s06_a10_r05.txt


In [13]:
print(x2.shape)
print(y2.shape)

(220, 6)
(220,)


In [14]:
x2

array([[ 0.04006436,  0.35017315,  0.12262123,  0.649134  , -0.33199771,
         0.26723573],
       [ 0.00772599,  0.42546477,  0.18102027,  0.33986785,  0.04540719,
         0.41435036],
       [-0.00688675,  0.38390344,  0.14738185,  0.10626748, -0.59637342,
         0.65674069],
       ...,
       [ 0.04240475,  0.12494149,  0.01561038,  1.91139415,  3.85545573,
         0.01499655],
       [ 0.02673548,  0.12504011,  0.01563503,  1.54896508,  2.69461657,
         0.01707883],
       [ 0.04412476,  0.14078769,  0.01982117,  1.15948   ,  0.56160031,
         0.02747787]])

In [15]:
y2

array(['01', '01', '01', '01', '01', '02', '02', '02', '02', '02', '03',
       '03', '03', '03', '03', '04', '04', '04', '04', '04', '05', '05',
       '05', '05', '05', '06', '06', '06', '06', '06', '07', '07', '07',
       '07', '07', '08', '08', '08', '08', '08', '09', '09', '09', '09',
       '09', '10', '10', '10', '10', '10', '11', '11', '11', '11', '11',
       '01', '01', '01', '01', '01', '02', '02', '02', '02', '02', '03',
       '03', '03', '03', '03', '04', '04', '04', '04', '04', '05', '05',
       '05', '05', '05', '06', '06', '06', '06', '06', '07', '07', '07',
       '07', '07', '08', '08', '08', '08', '08', '09', '09', '09', '09',
       '09', '10', '10', '10', '10', '10', '11', '11', '11', '11', '11',
       '01', '01', '01', '01', '01', '02', '02', '02', '02', '02', '03',
       '03', '03', '03', '03', '04', '04', '04', '04', '04', '05', '05',
       '05', '05', '05', '06', '06', '06', '06', '06', '07', '07', '07',
       '07', '07', '08', '08', '08', '08', '08', '0

In [17]:
m1 = x2[my_index_multi(y2, '01')].mean()
m2 = x2[my_index_multi(y2, '02')].mean()
m3 = x2[my_index_multi(y2, '03')].mean()
m4 = x2[my_index_multi(y2, '04')].mean()
m5 = x2[my_index_multi(y2, '05')].mean()
m6 = x2[my_index_multi(y2, '06')].mean()
m7 = x2[my_index_multi(y2, '07')].mean()
m8 = x2[my_index_multi(y2, '08')].mean()
m9 = x2[my_index_multi(y2, '09')].mean()
m10 = x2[my_index_multi(y2, '10')].mean()
m11 = x2[my_index_multi(y2, '11')].mean()
print(m1)
print(m2)
print(m3)
print(m4)
print(m5)
print(m6)
print(m7)
print(m8)
print(m9)
print(m10)
print(m11)

0.32206604544198847
0.1802285935481173
2.5809455760449738
2.1859622338249145
4.019760975052383
3.586730064165977
2.274620479227031
1.004176646107731
2.902198256595738
1.1754835719683723
0.9013805381954081


In [18]:
np.sqrt(((m1- 0.2837427766466512)**2+(m2- 0.1733721261595955)**2+(m3- 0.2416587854446975)**2//
+(m4- 1.7662228027076237)**2+(m5-5.252416738551928)**2+(m6-5.506056486778035)**2//
+(m7-2.3397725028515692)**2+(m8-1.1361369336776836)**2+(m9-1.1488882440431472)**2//
 +(m10-0.8139112866822661)**2+(m11-0.6847317601151889)**2)/11)

9.158132531580403

In [ ]:
0.2837427766466512
0.1733721261595955
0.2416587854446975
1.7662228027076237
5.252416738551928
5.506056486778035
2.3397725028515692
1.1361369336776836
1.1488882440431472
0.8139112866822661
0.6847317601151889

In [13]:
def ml_ar(X,y):
    x_train, x_test, y_train, y_test = crv.train_test_split(X, y, test_size=0.30, random_state=42)
    clf = RandomForestClassifier(n_estimators=20, random_state=42)
    clf.fit(x_train, y_train)
    y_predict = clf.predict(x_test)
    conf = confusion_matrix(y_test, y_predict)
    print (accuracy_score(y_test, y_predict) ) 
    actions = ["01","02","03","04","05","06","07","08","09","10","11"]
    print(classification_report(y_test, y_predict, target_names=actions))
    print(conf)

In [14]:
ml_ar(x2,y2)

0.5050505050505051
             precision    recall  f1-score   support

         01       0.45      0.45      0.45        22
         02       0.35      0.33      0.34        18
         03       0.62      0.65      0.63        20
         04       0.75      0.50      0.60        18
         05       0.62      0.80      0.70        20
         06       0.50      0.43      0.46        21
         07       0.78      0.93      0.85        15
         08       0.44      0.47      0.45        15
         09       0.31      0.31      0.31        13
         10       0.41      0.37      0.39        19
         11       0.28      0.29      0.29        17

avg / total       0.50      0.51      0.50       198

[[10 11  0  0  0  1  0  0  0  0  0]
 [12  6  0  0  0  0  0  0  0  0  0]
 [ 0  0 13  0  0  1  0  1  5  0  0]
 [ 0  0  1  9  0  0  1  1  3  1  2]
 [ 0  0  0  0 16  4  0  0  0  0  0]
 [ 0  0  0  0 10  9  1  0  1  0  0]
 [ 0  0  0  1  0  0 14  0  0  0  0]
 [ 0  0  1  0  0  0  1  7  0  1  5]
 

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

0.5050505050505051
             precision    recall  f1-score   support

         01       0.50      0.32      0.39        22
         02       0.42      0.61      0.50        18
         03       0.54      0.70      0.61        20
         04       0.75      0.50      0.60        18
         05       0.58      0.55      0.56        20
         06       0.44      0.33      0.38        21
         07       0.68      1.00      0.81        15
         08       0.40      0.53      0.46        15
         09       0.33      0.23      0.27        13
         10       0.41      0.37      0.39        19
         11       0.47      0.47      0.47        17

avg / total       0.51      0.51      0.49       198

[[ 7 15  0  0  0  0  0  0  0  0  0]
 [ 7 11  0  0  0  0  0  0  0  0  0]
 [ 0  0 14  0  0  1  1  1  3  0  0]
 [ 0  0  1  9  0  0  1  2  3  1  1]
 [ 0  0  1  0 11  6  2  0  0  0  0]
 [ 0  0  1  2  8  7  3  0  0  0  0]
 [ 0  0  0  0  0  0 15  0  0  0  0]
 [ 0  0  1  0  0  1  0  8  0  2  3]
 [ 0  0  8  0  0  1  0  1  3  0  0]
 [ 0  0  0  1  0  0  0  6  0  7  5]
 [ 0  0  0  0  0  0  0  2  0  7  8]]